In [ ]:
# carregar dataset rudementar


# ==============================================================
# COLETA DE POIs COMPLETA — Turismo + Infraestrutura
# Angola (Quedas de Calandula, Miradouro da Lua, Parque da Kissama)
# ==============================================================
import osmnx as ox
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from pathlib import Path

# === Diretório de saída ===
DATA_DIR = Path("../data/geoespacial")
DATA_DIR.mkdir(parents=True, exist_ok=True)

# === Locais turísticos ===
locais_turisticos = [
    {"nome": "Quedas de Calandula", "local": "Malanje, Angola"},
    {"nome": "Miradouro da Lua", "local": "Luanda, Angola"},
    {"nome": "Parque da Kissama", "local": "Luanda, Angola"}
]

BUFFER_RADIUS = 5000  # 5 km
dfs = []

# === Tags OSM — Turismo + Infraestrutura ===
TAGS = {
    "tourism": True,
    "amenity": True,
    "leisure": True,
    "natural": True,
    "landuse": True,
    "highway": True,
    "waterway": True,
    "power": True,       #  Linhas e subestações elétricas
    "man_made": True,    #  Estruturas construídas (pontes, torres, reservatórios)
    "railway": True,     #  Trilhos e estações de trem
    "building": True     #  Edificações
}

# === Loop pelos locais ===
for item in locais_turisticos:
    nome = item["nome"]
    local = item["local"]
    print(f"\n Coletando POIs em torno de {nome}...")

    try:
        #  Geocodificar o local
        ponto = ox.geocode(f"{nome}, {local}")
        ponto_geom = Point(ponto[::-1])
        print(f"    Coordenadas: ({ponto[0]:.6f}, {ponto[1]:.6f})")

        #  Criar buffer de busca
        gdf_ponto = gpd.GeoDataFrame([[nome, ponto_geom]], columns=["nome", "geometry"], crs="EPSG:4326")
        area_busca = gdf_ponto.to_crs(3857).buffer(BUFFER_RADIUS).to_crs(4326).iloc[0]

        #  Buscar POIs via OSM
        pois = ox.features_from_polygon(area_busca, TAGS).reset_index()

        #  Garantir colunas (evita KeyError)
        colunas_necessarias = [
            "osm_id", "name", "tourism", "amenity", "leisure",
            "natural", "landuse", "highway", "waterway",
            "power", "man_made", "railway", "building", "geometry"
        ]
        for c in colunas_necessarias:
            if c not in pois.columns:
                pois[c] = None

        pois = pois[colunas_necessarias]

        #  Adicionar metadados
        pois["ponto_turistico"] = nome
        pois["localidade"] = local
        pois["fonte_dados"] = "OpenStreetMap"
        pois["latitude"] = pois.to_crs(3857).geometry.centroid.to_crs(4326).y
        pois["longitude"] = pois.to_crs(3857).geometry.centroid.to_crs(4326).x

        #  Salvar individualmente
        output_path = DATA_DIR / f"pois_{nome.replace(' ', '_').lower()}.geojson"
        pois.to_file(output_path, driver="GeoJSON")
        print(f"   Salvo: {output_path}")

        # Mostrar resumo (5 primeiras linhas)
        print("\n Amostra de POIs encontrados:")
        display(
            pois.head(5)[
                ["name", "tourism", "amenity", "leisure", "natural",
                 "landuse", "highway", "power", "man_made", "railway", "building",
                 "latitude", "longitude"]
            ].fillna("—")
        )

        #  Mostrar categorias encontradas
        print("\n Categorias detectadas:")
        for col in ["tourism", "amenity", "leisure", "highway", "power", "man_made", "railway", "building"]:
            valores = pois[col].dropna().unique().tolist()
            if valores:
                print(f"   - {col}: {valores[:5]}")

        dfs.append(pois)

    except Exception as e:
        print(f" Erro ao processar {nome}: {e}")

# === Combinar tudo ===
if dfs:
    pois_final = gpd.GeoDataFrame(pd.concat(dfs, ignore_index=True), crs="EPSG:4326")
    output_final = DATA_DIR / "pontos_turisticos_angola_completos.geojson"
    pois_final.to_file(output_final, driver="GeoJSON")
    print(f"\n Arquivo final combinado salvo em: {output_final}")
else:
    print("\n Nenhum POI foi coletado.")




 Coletando POIs em torno de Quedas de Calandula...
    Coordenadas: (-9.074067, 16.000316)
   Salvo: ..\data\raw\pois_quedas_de_calandula.geojson

 Amostra de POIs encontrados:


,name,tourism,amenity,leisure,natural,landuse,highway,power,man_made,railway,building,latitude,longitude
0,Quedas de Calandula,—,—,—,—,—,—,—,—,—,—,-9.074067,16.000316
1,—,—,—,—,—,—,turning_circle,—,—,—,—,-9.074009,15.998896
2,—,—,—,—,—,—,—,—,—,—,—,-9.074206,16.002487
3,—,camp_site,—,—,—,—,—,—,—,—,—,-9.077999,16.002056
4,Estrada de Calandula,—,—,—,—,—,secondary,—,—,—,—,-9.111147,15.970919



 Categorias detectadas:
   - tourism: ['camp_site', 'picnic_site', 'hotel']
   - amenity: ['shelter', 'restaurant']
   - leisure: ['park', 'swimming_pool']
   - highway: ['turning_circle', 'secondary', 'unclassified', 'track', 'tertiary']
   - building: ['yes', 'roof', 'hotel']

 Coletando POIs em torno de Miradouro da Lua...
    Coordenadas: (-9.221175, 13.089830)
   Salvo: ..\data\raw\pois_miradouro_da_lua.geojson

 Amostra de POIs encontrados:


,name,tourism,amenity,leisure,natural,landuse,highway,power,man_made,railway,building,latitude,longitude
0,Miradouro da Lua,attraction,—,—,—,—,—,—,—,—,—,-9.221175,13.089830
1,—,—,—,—,—,—,—,—,antenna,—,—,-9.219571,13.090260
2,—,—,—,—,cliff,—,—,—,—,—,—,-9.262782,13.100828
3,—,—,—,—,cliff,—,—,—,—,—,—,-9.262581,13.100718
4,—,—,—,—,—,—,—,tower,—,—,—,-9.184286,13.068307



 Categorias detectadas:
   - tourism: ['attraction']
   - amenity: ['marketplace']
   - leisure: ['resort', 'swimming_pool']
   - highway: ['give_way', 'track', 'unclassified', 'primary', 'service']
   - power: ['tower', 'line', 'substation']
   - man_made: ['antenna', 'tower']
   - building: ['yes']

 Coletando POIs em torno de Parque da Kissama...
    Coordenadas: (-9.361165, 13.176456)
   Salvo: ..\data\raw\pois_parque_da_kissama.geojson

 Amostra de POIs encontrados:


,name,tourism,amenity,leisure,natural,landuse,highway,power,man_made,railway,building,latitude,longitude
0,Imbondeiro,—,restaurant,—,—,—,—,—,—,—,—,-9.337863,13.157969
1,—,—,restaurant,—,—,—,—,—,—,—,—,-9.342203,13.153666
2,—,—,—,—,—,—,—,—,surveillance,—,—,-9.322501,13.161966
3,Parque Nacional do Quiçama,attraction,—,—,—,—,—,—,—,—,—,-9.361165,13.176456
4,—,—,—,—,—,—,—,—,antenna,—,—,-9.395573,13.181716



 Categorias detectadas:
   - tourism: ['attraction']
   - amenity: ['restaurant']
   - leisure: ['resort', 'golf_course', 'pitch']
   - highway: ['primary', 'track', 'unclassified', 'service', 'residential']
   - man_made: ['surveillance', 'antenna', 'pier']
   - building: ['roof', 'yes']

 Arquivo final combinado salvo em: ..\data\raw\pontos_turisticos_angola_completos.geojson


In [ ]:
# Gerar variáveis espaciais


In [ ]:
# Cruzar camadas (join espacial)


In [ ]:
# Salvar dataset final


In [ ]:
# Entrega esperada: model_inputs.csv